<a href="https://colab.research.google.com/github/aiaaee/GoogLeNet/blob/main/GoogLeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch

In [24]:
class BasicConv2d(nn.Module):
  def __init__(self , in_channel=int  , out_channel=int , kernel_size=int , stride=int , padding=int , *args , **kwargs):
    super().__init__(*args , **kwargs)
    self.conv = nn.Conv2d(in_channels = in_channel , out_channels = out_channel , kernel_size=kernel_size , stride=stride , padding=padding, **kwargs)
    self.bn = nn.BatchNorm2d(out_channel)
    self.relu = nn.ReLU()

  def forward(self , x ):
    x = self.conv(x)
    x = self.bn(x)
    x = self.relu(x)
    return x


class Inception_Module(nn.Module):
  def __init__(self , in_channel , in_layer1_1 , in_layer2_1 , in_layer2_2,in_layer3_1 , in_layer3_2 , in_layer4_2) :
    super().__init__()
    self.b1 = BasicConv2d(in_channel , in_layer1_1 , kernel_size=1 , stride=1 , padding=0)
    self.b2 = nn.Sequential(
        BasicConv2d(in_channel , in_layer2_1 , kernel_size=1, stride=1 , padding=0),
        BasicConv2d(in_layer2_1 , in_layer2_2 , kernel_size=3, stride=1 , padding=1),
    )
    self.b3 = nn.Sequential(
        BasicConv2d(in_channel , in_layer3_1 , kernel_size=1 ,  stride=1 , padding=0) ,
        BasicConv2d(in_layer3_1 , in_layer3_2 , kernel_size=5 ,  stride=1 , padding=2)
    )
    self.b4 = nn.Sequential(
        nn.MaxPool2d(kernel_size=(3,3) , stride=1 , padding=1) ,
        BasicConv2d(in_channel , in_layer4_2 , kernel_size=1 , stride=1 , padding=0)
    )

  def forward(self,x ):
    b1 = self.b1(x)
    b2 = self.b2(x)
    b3 = self.b3(x)
    b4 = self.b4(x)
    x =  torch.cat([b1 , b2 , b3 , b4 ] , 1 )
    return x

In [85]:
class GoogLeNet(nn.Module):
  def __init__(self , in_channel , output , *args , **kwargs):
    super().__init__(*args , **kwargs)
    self.conv1 = BasicConv2d(in_channel , 64 , kernel_size=7 , stride=2 , padding=3)
    self.maxpool1 = nn.MaxPool2d(kernel_size=(3,3) , stride=2 , padding=1)
    self.lrn1 = nn.LocalResponseNorm(size=5 , alpha=0.0001 , beta=0.75 , k=2)
    self.conv2 = BasicConv2d(64 , 64 , kernel_size=1 , stride=1 , padding=0)
    self.conv3 = BasicConv2d(64 , 192 , kernel_size=3 , stride=1 , padding=2)
    self.lrn2 = nn.LocalResponseNorm(size=5 , alpha=0.0001 , beta=0.75 , k=2)
    self.maxpool2 = nn.MaxPool2d(kernel_size=3 , stride=2 , padding=1)
    self.inception1 = Inception_Module(192 , 64 , 96 , 128 , 12 , 32 , 32 )
    self.inception2 = Inception_Module(256 , 128 , 128 , 192 , 32 , 96 , 64)
    self.maxpool3 = nn.MaxPool2d(kernel_size=3 , stride=2 , padding=1)
    self.inception3 = Inception_Module(480 , 192 , 96 , 208 , 16 , 48 , 64)
    self.inception4 = Inception_Module(512 , 160 , 112 , 224 , 24 , 64 , 64 )
    self.inception5 = Inception_Module(512 , 128 , 128 , 256 , 24 , 64 , 64)
    self.inception6 = Inception_Module(512 , 112 , 144 , 288 , 32 , 64 , 64 )
    self.inception7 = Inception_Module(528 ,  256 , 160  ,320 , 32 , 128 , 128)
    self.maxpool4 = nn.MaxPool2d(kernel_size=3 , stride=2 , padding=1)
    self.inception8 = Inception_Module(832 , 256 , 160 , 320, 32 , 128 , 128)
    self.inception9 = Inception_Module(832 , 384 , 192 , 384 , 48 , 128 , 128)
    self.gavgpool = nn.AdaptiveAvgPool2d((1 , 1 ))
    self.flat = nn.Flatten()
    self.dropout = nn.Dropout2d(0.4)
    self.classifier1 = nn.Linear(832 , 1000)
    self.classifier2 = nn.Linear(1000 , output)
  def forward(self , x ):
    x = self.conv1(x)
    x = self.maxpool1(x)
    x = self.lrn1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.lrn2(x)
    x = self.maxpool2(x)
    x = self.inception1(x)
    x = self.inception2(x)
    x = self.maxpool3(x)
    x = self.inception3(x)
    x = self.inception4(x)
    x = self.inception5(x)
    x = self.inception6(x)
    x = self.inception7(x)
    x = self.maxpool4(x)
    x = self.inception8(x)
    x = self.gavgpool(x)
    x = self.flat(x)
    x = self.dropout(x)
    x = self.classifier1(x)
    x = self.classifier2(x)
    return  x




In [86]:
x = torch.randn(32 , 3 , 224 , 224)
model = GoogLeNet(3 , 10)
model(x).size()

torch.Size([32, 10])